In [5]:
import duckdb
import pandas as pd
import numpy as np
import re

# 1. 加载 Excel 文件
excel_path = "Research Data sent.xlsx"
df = pd.read_excel(excel_path, sheet_name="Consolidated")

print("=== 数据清理和导入 ===\n")
print(f"原始数据: {len(df)} 行, {len(df.columns)} 列")

# 2. 清理列名
df.columns = [c.strip().replace(' ', '_').replace('(', '').replace(')', '') for c in df.columns]

# 3. 数据清理函数
def clean_stock_and_inventory(column):
    """清理 Stock on Market 和 Months of Inventory 列"""
    # 将 'Not Available' 替换为 NaN
    cleaned = column.replace('Not Available', np.nan)
    # 转换为数值类型
    return pd.to_numeric(cleaned, errors='coerce')

def clean_distance_to_cbd(column):
    """清理 Distance to Nearest CBD 列，提取数值部分"""
    def extract_number(value):
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            # 提取数字部分，支持小数
            match = re.search(r'(\d+\.?\d*)', value)
            if match:
                return float(match.group(1))
        return np.nan
    
    return column.apply(extract_number)

# 4. 执行数据清理
print("🧹 清理数据...")

# 清理 Stock on Market
if 'Stock_on_Market' in df.columns:
    original_type = df['Stock_on_Market'].dtype
    df['Stock_on_Market'] = clean_stock_and_inventory(df['Stock_on_Market'])
    print(f"  ✅ Stock_on_Market: {original_type} → {df['Stock_on_Market'].dtype}")
    print(f"     空值: {df['Stock_on_Market'].isnull().sum()} ({df['Stock_on_Market'].isnull().sum()/len(df)*100:.1f}%)")

# 清理 Months of Inventory
if 'Months_of_Inventory' in df.columns:
    original_type = df['Months_of_Inventory'].dtype
    df['Months_of_Inventory'] = clean_stock_and_inventory(df['Months_of_Inventory'])
    print(f"  ✅ Months_of_Inventory: {original_type} → {df['Months_of_Inventory'].dtype}")
    print(f"     空值: {df['Months_of_Inventory'].isnull().sum()} ({df['Months_of_Inventory'].isnull().sum()/len(df)*100:.1f}%)")

# 清理 Distance to Nearest CBD
if 'Distance_to_Nearest_CBD' in df.columns:
    original_type = df['Distance_to_Nearest_CBD'].dtype
    df['Distance_to_Nearest_CBD'] = clean_distance_to_cbd(df['Distance_to_Nearest_CBD'])
    print(f"  ✅ Distance_to_Nearest_CBD: {original_type} → {df['Distance_to_Nearest_CBD'].dtype}")
    print(f"     空值: {df['Distance_to_Nearest_CBD'].isnull().sum()} ({df['Distance_to_Nearest_CBD'].isnull().sum()/len(df)*100:.1f}%)")

# 清理其他 'Not Available' 值
print("\n🔍 检查其他 'Not Available' 值...")
for col in df.columns:
    if df[col].dtype == 'object':
        not_available_count = (df[col] == 'Not Available').sum()
        if not_available_count > 0:
            print(f"  - {col}: {not_available_count} 个 'Not Available'")
            # 如果是数值列，尝试转换
            if col not in ['Suburb_Name', 'Local_Government_Area', 'State']:
                df[col] = df[col].replace('Not Available', np.nan)
                # 尝试转换为数值
                numeric_series = pd.to_numeric(df[col], errors='coerce')
                if not numeric_series.isnull().all():
                    df[col] = numeric_series
                    print(f"    → 已转换为数值类型")

# 5. 创建 DuckDB 文件
print(f"\n💾 写入数据库...")
con = duckdb.connect("suburb_data.duckdb")
con.register("temp_df", df)
con.execute("CREATE OR REPLACE TABLE suburbs AS SELECT * FROM temp_df")

print("✅ 成功创建 suburb_data_cleaned.duckdb 并写入 suburbs 表")

# 6. 验证结果
print(f"\n🔬 验证清理结果:")
schema_info = con.execute("DESCRIBE suburbs").fetchdf()
print("关键列的数据类型:")
for col in ['Stock_on_Market', 'Months_of_Inventory', 'Distance_to_Nearest_CBD']:
    if col in schema_info['column_name'].values:
        dtype = schema_info[schema_info['column_name'] == col]['column_type'].iloc[0]
        print(f"  {col}: {dtype}")

con.close()
print("\n🎉 数据清理完成！") 

=== 数据清理和导入 ===

原始数据: 8759 行, 25 列
🧹 清理数据...
  ✅ Stock_on_Market: object → float64
     空值: 80 (0.9%)
  ✅ Months_of_Inventory: object → float64
     空值: 80 (0.9%)
  ✅ Distance_to_Nearest_CBD: object → float64
     空值: 0 (0.0%)

🔍 检查其他 'Not Available' 值...
  - Vacancy_Rate: 5457 个 'Not Available'
    → 已转换为数值类型
  - Renters: 1426 个 'Not Available'
    → 已转换为数值类型

💾 写入数据库...
✅ 成功创建 suburb_data_cleaned.duckdb 并写入 suburbs 表

🔬 验证清理结果:
关键列的数据类型:
  Stock_on_Market: DOUBLE
  Months_of_Inventory: DOUBLE
  Distance_to_Nearest_CBD: DOUBLE

🎉 数据清理完成！


/var/folders/b4/ww85fx6974s82fzdgclxx86m0000gn/T/ipykernel_52051/2090734273.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned = column.replace('Not Available', np.nan)
/var/folders/b4/ww85fx6974s82fzdgclxx86m0000gn/T/ipykernel_52051/2090734273.py:71: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('Not Available', np.nan)


In [6]:
# 查 top 5 房价最高的 suburb
con.execute("""
SELECT *
FROM suburbs
ORDER BY Smart_Median_House_Price DESC
LIMIT 5
""").fetchdf()

Suburb_Name Local_Government_Area            State  \
0       Vaucluse             Woollahra  New South Wales   
1    Point Piper             Woollahra  New South Wales   
2    Watsons Bay             Woollahra  New South Wales   
3  Bellevue Hill             Woollahra  New South Wales   
4       Tamarama              Waverley  New South Wales   

   Smart_Median_House_Price  Gross_Yield  Rent  \
0                   7410000        0.024  3100   
1                   6820000        0.033  3010   
2                   6420000        0.024  3010   
3                   6350000        0.021  2370   
4                   5600000        0.022  2270   

   Annualized_growth_Forecast_next_four_years  10_Year_Growth_2014-2024  \
0                                       0.058                     0.087   
1                                       0.078                     0.101   
2                                       0.078                     0.085   
3                                       0.055                     0.092   
4                                       0.078                     0.103   

   3_Year_Growth_2021-2024 Stock_on_Market  ... Welfare_Reliance  \
0                    0.043              30  ...            0.086   
1                    0.056               2  ...            0.068   
2                    0.028               1  ...            0.101   
3                   -0.009              38  ...            0.077   
4                    0.027               2  ...            0.059   

   Unemployment_Rate Single_Parent_Households_Percentage  Family_Households  \
0              0.000                               0.070              0.751   
1              0.002                               0.081              0.675   
2              0.003                               0.069              0.674   
3              0.000                               0.066              0.678   
4              0.003                               0.040              0.690   

   Total_Private_School_Students  Median_Household_Income_Weekly  \
0                           1310                            3420   
1                            111                            4530   
2                             93                            2920   
3                           1370                            3320   
4                            111                            4520   

   Affluence_Score   Vacancy_Rate  Renters  Distance_to_State_CBD_km  
0            0.991          0.009    0.193                       6.3  
1            0.996  Not Available      0.2                       3.8  
2            0.966  Not Available    0.129                       6.3  
3            0.995          0.008    0.291                       4.7  
4            0.985          0.005    0.329                       6.0  

[5 rows x 25 columns]